In [4]:
import numpy as np
import pandas as pd
from pgmpy.readwrite import XMLBIFReader
import random
import copy

# Settings
from discriminativeBN import DiscriminativeBN
from simplexMethod import NelderMeadOptimizer
from hookeJeeves import HookeJeevesOptimizer
from kfold_runner import k_fold_validation
from kfold_runner_lbfgs import k_fold_validation_gradient
from lbfgs import LBFGSOptimizerWrapper

import time
import logging

In [5]:
data_configs = [
    {    "name": "Australian",
        "csv": "datasets/AustralianDisc.csv",
        "xml": "models/australianTAN.xml",
        "target_var": "A15" 
    }
]

optimizers = [
    {
        "name": "Nelder-Mead",
        "func": k_fold_validation,            # Standard function
        "Optimizer_fun": NelderMeadOptimizer,
        "max_iter": 5, #For testing
        "file_suffix": "simplex"
    },
    {
        "name": "Hooke-Jeeves",
        "func": k_fold_validation,            # Standard function
        "Optimizer_fun": HookeJeevesOptimizer,
        "max_iter": 3, #for testing
        "file_suffix": "hooke"
    },
    {
        "name": "L-BFGS (Gradient)",
        "func": k_fold_validation_gradient,   # Gradient specific function
        "Optimizer_fun": LBFGSOptimizerWrapper,
        "max_iter": 100,
        "file_suffix": "LBFGS"
    }
]

In [6]:

# 3. BUILD: Generate the list of experiments dynamically
experiments = []

for data in data_configs:
    for opt in optimizers:
        
        # Auto-generate the results filename based on dataset and method
        # Example: results_AustralianTAN_simplex.txt
        result_name = f"resultsDemo/results_{data['name']}NAIVE_{opt['file_suffix']}.txt"
        
        experiments.append({
            "name": f"{data['name']} - {opt['name']}",
            "func": opt['func'],
            "params": {
                "xml_file": data['xml'],
                "csv_file": data['csv'],
                "target_var": data['target_var'],
                "Optimizer_fun": opt['Optimizer_fun'],
                "max_iter": opt['max_iter'],
                "results_file": result_name
            }
        })

# 4. EXECUTE: Run the loop
print(f"Scheduled {len(experiments)} experiments.\n")

for exp in experiments:
    print(f"--- Starting Experiment: {exp['name']} ---")
    start_time = time.time()
    
    try:
        # Unpack the dictionary parameters into the function
        exp['func'](**exp['params'])
        
    except Exception as e:
        print(f"CRITICAL ERROR in {exp['name']}: {e}")
        logging.error(f"Failed {exp['name']}: {e}")
        # 'continue' jumps to the next iteration of the for-loop
        continue 
    
    else:
        # This only runs if the 'try' block succeeded
        end_time = time.time()
        elapsed_minutes = (end_time - start_time) / 60
        print(f"Successfully finished {exp['name']}.")
        print(f"Saved to: {exp['params']['results_file']}")
        print(f"Elapsed time: {elapsed_minutes:.2f} minutes\n")

print("\nAll scheduled experiments have been processed.")

Scheduled 3 experiments.

--- Starting Experiment: Australian - Nelder-Mead ---
Starting 10-Fold Validation on datasets/AustralianDisc.csv...
Optimizer: NelderMeadOptimizer | Max Iter: 5
--- Starting Simplex Optimization (Dim: 118) ---
Initial Best Score: -415.0223 (CLL)
Initial Worst Score: -423.0590 (CLL)
------------------------------
------------------------------
Optimization Finished. Best Score found: -411.0783
1 fold 59.42%
--- Starting Simplex Optimization (Dim: 118) ---
Initial Best Score: -391.3324 (CLL)
Initial Worst Score: -398.1020 (CLL)
------------------------------
------------------------------
Optimization Finished. Best Score found: -387.9874
2 fold 75.36%
--- Starting Simplex Optimization (Dim: 118) ---
Initial Best Score: -476.7947 (CLL)
Initial Worst Score: -486.1746 (CLL)
------------------------------
------------------------------
Optimization Finished. Best Score found: -472.1762
3 fold 31.88%
--- Starting Simplex Optimization (Dim: 118) ---
Initial Best Scor